In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import datetime
import time
from urllib.parse import quote
import random
import pandas as pd
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import sent_tokenize
from nltk.probability import FreqDist

Collect Data

In [34]:
def google_news_scraper(query,start_date,end_date,WebDriver_path):
    chromedriver_path = WebDriver_path
    # Configure Chrome options
    options = Options()
    options.headless = False  # Set to True to run Chrome in headless mode (no visible window)
    # Set the path to chromedriver using the Service object
    service = Service(chromedriver_path)
    # Initialize Chrome webdriver with service and options
    driver = webdriver.Chrome(service=service, options=options)
    query_string = quote(query)
    url = f"https://www.google.com/search?q={query_string}&tbm=nws&tbs=cdr:1,cd_min:{start_date},cd_max:{end_date}"
    driver.get(url)
    # Check if a CAPTCHA page is loaded
    if "sorry/index?continue" in driver.current_url:
        print("CAPTCHA encountered. Please solve it manually.") # Pause the script and wait for the user to solve the CAPTCHA manually
        input("Press 'Enter' after solving the CAPTCHA...")  # Wait for user input after CAPTCHA solved        
    # Print the current page title and URL for debugging
    print("Page Title:", driver.title)
    print("Current URL:", driver.current_url)
    # Wait for a few seconds to allow the page to load completely
    random_delay = random.randint(3, 10)  # Generate a random delay between 3 to 10 seconds
    time.sleep(random_delay)
    # Finding the news elements using CSS selector (class name of the google element will change in the future)
    news_results = driver.find_elements(By.CLASS_NAME, "AZtVbe")
    news_results.extend(driver.find_elements(By.CLASS_NAME, "SoaBEf"))
    news_results.extend(driver.find_elements(By.CLASS_NAME, "CBAQAA"))
    news_results.extend(driver.find_elements(By.CLASS_NAME, "SoaBEf.R24aHf"))
    # The list contains the resulting dictionaries
    finalResult = []
    if len(news_results) >= 10: 
        for news_div in news_results[0:10]: # Get exactly 10 news from google news first page
            try:
                # Extracting necessary information
                news_link = news_div.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
                news_title = news_div.find_element(By.CSS_SELECTOR, '.n0jPhd.ynAwRc.MBeuO.nDgy9d').text
                news_snippet = news_div.find_element(By.CSS_SELECTOR, '.GI74Re.nDgy9d').text
                news_date = news_div.find_element(By.CSS_SELECTOR, '.OSrXXb.rbYSKb.LfVVr').text
                # append the result into list of dictionaries
                finalResult.append({
                    "Title": news_title,
                    "URL": news_link,
                    "Snippet": news_snippet,
                    "Post at": news_date
                })
            except Exception as e:
                print("No elements found")
            time.sleep(random_delay)  # Sleep for the random time interval before the next iteration
    else:
        print("try other query and date") # not enough 10 news
    driver.quit()
    return finalResult

def news_contents(url,snippet,query,key_search,webdriver_path):
    chromedriver_path = webdriver_path    
    options = Options()
    options.headless = True  # Set to True to run Chrome in headless mode (no visible window)    
    service = Service(chromedriver_path)
    driver = webdriver.Chrome(service=service, options=options)    
    random_delay = random.randint(1,2)
    try:
        driver.get(url)    
        # Wait for the page to load completely
        time.sleep(2) 
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')        
        contents = soup.find("body")
        footer = soup.find("footer")
        if footer:
            footer.extract()        
        all_contents = []
        for content in contents.find_all("p"):
            all_contents.append(str(content.get_text()))
            time.sleep(1)        
        full_content = ' '.join(all_contents)
        if query.lower() in full_content.lower():
            return full_content
        elif key_search in full_content:
            return full_content
        else:
            return snippet        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return snippet    
    finally:
        driver.quit()

def extractive_summary(text, vietnamese_stopwords,num_sentences=10):
    if len(text) > 200:
        sentences = sent_tokenize(text)
        words = nltk.word_tokenize(text.lower())
        filtered_words = [word for word in words if word.isalnum() and word not in vietnamese_stopwords]
        freq_dist = FreqDist(filtered_words)
        sentence_scores = {}
        for sentence in sentences:
            for word in nltk.word_tokenize(sentence.lower()):
                if word in freq_dist.keys():
                    if len(sentence.split(' ')) < 30:
                        if sentence not in sentence_scores.keys():
                            sentence_scores[sentence] = freq_dist[word]
                        else:
                            sentence_scores[sentence] += freq_dist[word]
        summary_sentences = sorted(sentence_scores.items(), key=lambda x: x[1], reverse=True)[:num_sentences]
        summary_sentences = sorted(summary_sentences, key=lambda x: sentences.index(x[0]))
        summary = ' '.join([sentence[0] for sentence in summary_sentences])
        return summary
    else:
        return text

def date_format(date): # use to change format
    return ''.join(date[3:5]+'/'+date[0:2]+'/'+date[6:])

random_delay_query = random.randint(1, 2)
webdriver_path = r'C:\WebDriver\chromedriver-win64\chromedriver.exe'
key_search = input("What do you want from Google News?\n")
start = input("Start date (dd/mm/yyyy)\n")
end = input("End date (dd/mm/yyyy)\n")
company = pd.read_csv('companyname_code.csv')
company = company.drop(index=0)
query = key_search

VNM = """a lô
a ha
ai
ai ai
ai nấy
ai đó
alô
amen
anh
anh ấy
ba
ba ba
ba bản
ba cùng
ba họ
ba ngày
ba ngôi
ba tăng
bao giờ
bao lâu
bao nhiêu
bao nả
bay biến
biết
biết bao
biết bao nhiêu
biết chắc
biết chừng nào
biết mình
biết mấy
biết thế
biết trước
biết việc
biết đâu
biết đâu chừng
biết đâu đấy
biết được
buổi
buổi làm
buổi mới
buổi ngày
buổi sớm
bà
bà ấy
bài
bài bác
bài bỏ
bài cái
bác
bán
bán cấp
bán dạ
bán thế
bây bẩy
bây chừ
bây giờ
bây nhiêu
bèn
béng
bên
bên bị
bên có
bên cạnh
bông
bước
bước khỏi
bước tới
bước đi
bạn
bản
bản bộ
bản riêng
bản thân
bản ý
bất chợt
bất cứ
bất giác
bất kì
bất kể
bất kỳ
bất luận
bất ngờ
bất nhược
bất quá
bất quá chỉ
bất thình lình
bất tử
bất đồ
bấy
bấy chầy
bấy chừ
bấy giờ
bấy lâu
bấy lâu nay
bấy nay
bấy nhiêu
bập bà bập bõm
bập bõm
bắt đầu
bắt đầu từ
bằng
bằng cứ
bằng không
bằng người
bằng nhau
bằng như
bằng nào
bằng nấy
bằng vào
bằng được
bằng ấy
bển
bệt
bị
bị chú
bị vì
bỏ
bỏ bà
bỏ cha
bỏ cuộc
bỏ không
bỏ lại
bỏ mình
bỏ mất
bỏ mẹ
bỏ nhỏ
bỏ quá
bỏ ra
bỏ riêng
bỏ việc
bỏ xa
bỗng
bỗng chốc
bỗng dưng
bỗng không
bỗng nhiên
bỗng nhưng
bỗng thấy
bỗng đâu
bộ
bộ thuộc
bộ điều
bội phần
bớ
bởi
bởi ai
bởi chưng
bởi nhưng
bởi sao
bởi thế
bởi thế cho nên
bởi tại
bởi vì
bởi vậy
bởi đâu
bức
cao
cao lâu
cao ráo
cao răng
cao sang
cao số
cao thấp
cao thế
cao xa
cha
cha chả
chao ôi
chia sẻ
chiếc
cho
cho biết
cho chắc
cho hay
cho nhau
cho nên
cho rằng
cho rồi
cho thấy
cho tin
cho tới
cho tới khi
cho về
cho ăn
cho đang
cho được
cho đến
cho đến khi
cho đến nỗi
choa
chu cha
chui cha
chung
chung cho
chung chung
chung cuộc
chung cục
chung nhau
chung qui
chung quy
chung quy lại
chung ái
chuyển
chuyển tự
chuyển đạt
chuyện
chuẩn bị
chành chạnh
chí chết
chính
chính bản
chính giữa
chính là
chính thị
chính điểm
chùn chùn
chùn chũn
chú
chú dẫn
chú khách
chú mày
chú mình
chúng
chúng mình
chúng ta
chúng tôi
chúng ông
chăn chắn
chăng
chăng chắc
chăng nữa
chơi
chơi họ
chưa
chưa bao giờ
chưa chắc
chưa có
chưa cần
chưa dùng
chưa dễ
chưa kể
chưa tính
chưa từng
chầm chập
chậc
chắc
chắc chắn
chắc dạ
chắc hẳn
chắc lòng
chắc người
chắc vào
chắc ăn
chẳng lẽ
chẳng những
chẳng nữa
chẳng phải
chết nỗi
chết thật
chết tiệt
chỉ
chỉ chính
chỉ có
chỉ là
chỉ tên
chỉn
chị
chị bộ
chị ấy
chịu
chịu chưa
chịu lời
chịu tốt
chịu ăn
chọn
chọn bên
chọn ra
chốc chốc
chớ
chớ chi
chớ gì
chớ không
chớ kể
chớ như
chợt
chợt nghe
chợt nhìn
chủn
chứ
chứ ai
chứ còn
chứ gì
chứ không
chứ không phải
chứ lại
chứ lị
chứ như
chứ sao
coi bộ
coi mòi
con
con con
con dạ
con nhà
con tính
cu cậu
cuối
cuối cùng
cuối điểm
cuốn
cuộc
càng
càng càng
càng hay
cá nhân
các
các cậu
cách
cách bức
cách không
cách nhau
cách đều
cái
cái gì
cái họ
cái đã
cái đó
cái ấy
câu hỏi
cây
cây nước
còn
còn như
còn nữa
còn thời gian
còn về
có
có ai
có chuyện
có chăng
có chăng là
có chứ
có cơ
có dễ
có họ
có khi
có ngày
có người
có nhiều
có nhà
có phải
có số
có tháng
có thế
có thể
có vẻ
có ý
có ăn
có điều
có điều kiện
có đáng
có đâu
có được
cóc khô
cô
cô mình
cô quả
cô tăng
cô ấy
công nhiên
cùng
cùng chung
cùng cực
cùng nhau
cùng tuổi
cùng tột
cùng với
cùng ăn
căn
căn cái
căn cắt
căn tính
cũng
cũng như
cũng nên
cũng thế
cũng vậy
cũng vậy thôi
cũng được
cơ
cơ chỉ
cơ chừng
cơ cùng
cơ dẫn
cơ hồ
cơ hội
cơ mà
cơn
cả
cả nghe
cả nghĩ
cả ngày
cả người
cả nhà
cả năm
cả thảy
cả thể
cả tin
cả ăn
cả đến
cảm thấy
cảm ơn
cấp
cấp số
cấp trực tiếp
cần
cần cấp
cần gì
cần số
cật lực
cật sức
cậu
cổ lai
cụ thể
cụ thể là
cụ thể như
của
của ngọt
của tin
cứ
cứ như
cứ việc
cứ điểm
cực lực
do
do vì
do vậy
do đó
duy
duy chỉ
duy có
dài
dài lời
dài ra
dành
dành dành
dào
dì
dù
dù cho
dù dì
dù gì
dù rằng
dù sao
dùng
dùng cho
dùng hết
dùng làm
dùng đến
dưới
dưới nước
dạ
dạ bán
dạ con
dạ dài
dạ dạ
dạ khách
dần dà
dần dần
dầu sao
dẫn
dẫu
dẫu mà
dẫu rằng
dẫu sao
dễ
dễ dùng
dễ gì
dễ khiến
dễ nghe
dễ ngươi
dễ như chơi
dễ sợ
dễ sử dụng
dễ thường
dễ thấy
dễ ăn
dễ đâu
dở chừng
dữ
dữ cách
em
em em
giá trị
giá trị thực tế
giảm
giảm chính
giảm thấp
giảm thế
giống
giống người
giống nhau
giống như
giờ
giờ lâu
giờ này
giờ đi
giờ đây
giờ đến
giữ
giữ lấy
giữ ý
giữa
giữa lúc
gây
gây cho
gây giống
gây ra
gây thêm
gì
gì gì
gì đó
gần
gần bên
gần hết
gần ngày
gần như
gần xa
gần đây
gần đến
gặp
gặp khó khăn
gặp phải
gồm
hay
hay biết
hay hay
hay không
hay là
hay làm
hay nhỉ
hay nói
hay sao
hay tin
hay đâu
hiểu
hiện nay
hiện tại
hoàn toàn
hoặc
hoặc là
hãy
hãy còn
hơn
hơn cả
hơn hết
hơn là
hơn nữa
hơn trước
hầu hết
hết
hết chuyện
hết cả
hết của
hết nói
hết ráo
hết rồi
hết ý
họ
họ gần
họ xa
hỏi
hỏi lại
hỏi xem
hỏi xin
hỗ trợ
khi
khi khác
khi không
khi nào
khi nên
khi trước
khiến
khoảng
khoảng cách
khoảng không
khá
khá tốt
khác
khác gì
khác khác
khác nhau
khác nào
khác thường
khác xa
khách
khó
khó biết
khó chơi
khó khăn
khó làm
khó mở
khó nghe
khó nghĩ
khó nói
khó thấy
khó tránh
không
không ai
không bao giờ
không bao lâu
không biết
không bán
không chỉ
không còn
không có
không có gì
không cùng
không cần
không cứ
không dùng
không gì
không hay
không khỏi
không kể
không ngoài
không nhận
không những
không phải
không phải không
không thể
không tính
không điều kiện
không được
không đầy
không để
khẳng định
khỏi
khỏi nói
kể
kể cả
kể như
kể tới
kể từ
liên quan
loại
loại từ
luôn
luôn cả
luôn luôn
luôn tay
là
là cùng
là là
là nhiều
là phải
là thế nào
là vì
là ít
làm
làm bằng
làm cho
làm dần dần
làm gì
làm lòng
làm lại
làm lấy
làm mất
làm ngay
làm như
làm nên
làm ra
làm riêng
làm sao
làm theo
làm thế nào
làm tin
làm tôi
làm tăng
làm tại
làm tắp lự
làm vì
làm đúng
làm được
lâu
lâu các
lâu lâu
lâu nay
lâu ngày
lên
lên cao
lên cơn
lên mạnh
lên ngôi
lên nước
lên số
lên xuống
lên đến
lòng
lòng không
lúc
lúc khác
lúc lâu
lúc nào
lúc này
lúc sáng
lúc trước
lúc đi
lúc đó
lúc đến
lúc ấy
lý do
lượng
lượng cả
lượng số
lượng từ
lại
lại bộ
lại cái
lại còn
lại giống
lại làm
lại người
lại nói
lại nữa
lại quả
lại thôi
lại ăn
lại đây
lấy
lấy có
lấy cả
lấy giống
lấy làm
lấy lý do
lấy lại
lấy ra
lấy ráo
lấy sau
lấy số
lấy thêm
lấy thế
lấy vào
lấy xuống
lấy được
lấy để
lần
lần khác
lần lần
lần nào
lần này
lần sang
lần sau
lần theo
lần trước
lần tìm
lớn
lớn lên
lớn nhỏ
lời
lời chú
lời nói
mang
mang lại
mang mang
mang nặng
mang về
muốn
mà
mà cả
mà không
mà lại
mà thôi
mà vẫn
mình
mạnh
mất
mất còn
mọi
mọi giờ
mọi khi
mọi lúc
mọi người
mọi nơi
mọi sự
mọi thứ
mọi việc
mối
mỗi
mỗi lúc
mỗi lần
mỗi một
mỗi ngày
mỗi người
một
một cách
một cơn
một khi
một lúc
một số
một vài
một ít
mới
mới hay
mới rồi
mới đây
mở
mở mang
mở nước
mở ra
mợ
mức
nay
ngay
ngay bây giờ
ngay cả
ngay khi
ngay khi đến
ngay lúc
ngay lúc này
ngay lập tức
ngay thật
ngay tức khắc
ngay tức thì
ngay từ
nghe
nghe chừng
nghe hiểu
nghe không
nghe lại
nghe nhìn
nghe như
nghe nói
nghe ra
nghe rõ
nghe thấy
nghe tin
nghe trực tiếp
nghe đâu
nghe đâu như
nghe được
nghen
nghiễm nhiên
nghĩ
nghĩ lại
nghĩ ra
nghĩ tới
nghĩ xa
nghĩ đến
nghỉm
ngoài
ngoài này
ngoài ra
ngoài xa
ngoải
nguồn
ngày
ngày càng
ngày cấp
ngày giờ
ngày ngày
ngày nào
ngày này
ngày nọ
ngày qua
ngày rày
ngày tháng
ngày xưa
ngày xửa
ngày đến
ngày ấy
ngôi
ngôi nhà
ngôi thứ
ngõ hầu
ngăn ngắt
ngươi
người
người hỏi
người khác
người khách
người mình
người nghe
người người
người nhận
ngọn
ngọn nguồn
ngọt
ngồi
ngồi bệt
ngồi không
ngồi sau
ngồi trệt
ngộ nhỡ
nhanh
nhanh lên
nhanh tay
nhau
nhiên hậu
nhiều
nhiều ít
nhiệt liệt
nhung nhăng
nhà
nhà chung
nhà khó
nhà làm
nhà ngoài
nhà ngươi
nhà tôi
nhà việc
nhân dịp
nhân tiện
nhé
nhìn
nhìn chung
nhìn lại
nhìn nhận
nhìn theo
nhìn thấy
nhìn xuống
nhóm
nhón nhén
như
như ai
như chơi
như không
như là
như nhau
như quả
như sau
như thường
như thế
như thế nào
như thể
như trên
như trước
như tuồng
như vậy
như ý
nhưng
nhưng mà
nhược bằng
nhất
nhất loạt
nhất luật
nhất là
nhất mực
nhất nhất
nhất quyết
nhất sinh
nhất thiết
nhất thì
nhất tâm
nhất tề
nhất đán
nhất định
nhận
nhận biết
nhận họ
nhận làm
nhận nhau
nhận ra
nhận thấy
nhận việc
nhận được
nhằm
nhằm khi
nhằm lúc
nhằm vào
nhằm để
nhỉ
nhỏ
nhỏ người
nhớ
nhớ bập bõm
nhớ lại
nhớ lấy
nhớ ra
nhờ
nhờ chuyển
nhờ có
nhờ nhờ
nhờ đó
nhỡ ra
những
những ai
những khi
những là
những lúc
những muốn
những như
nào
nào cũng
nào hay
nào là
nào phải
nào đâu
nào đó
này
này nọ
nên
nên chi
nên chăng
nên làm
nên người
nên tránh
nó
nóc
nói
nói bông
nói chung
nói khó
nói là
nói lên
nói lại
nói nhỏ
nói phải
nói qua
nói ra
nói riêng
nói rõ
nói thêm
nói thật
nói toẹt
nói trước
nói tốt
nói với
nói xa
nói ý
nói đến
nói đủ
năm
năm tháng
nơi
nơi nơi
nước
nước bài
nước cùng
nước lên
nước nặng
nước quả
nước xuống
nước ăn
nước đến
nấy
nặng
nặng căn
nặng mình
nặng về
nếu
nếu có
nếu cần
nếu không
nếu mà
nếu như
nếu thế
nếu vậy
nếu được
nền
nọ
nớ
nức nở
nữa
nữa khi
nữa là
nữa rồi
oai oái
oái
pho
phè
phè phè
phía
phía bên
phía bạn
phía dưới
phía sau
phía trong
phía trên
phía trước
phóc
phót
phù hợp
phăn phắt
phương chi
phải
phải biết
phải chi
phải chăng
phải cách
phải cái
phải giờ
phải khi
phải không
phải lại
phải lời
phải người
phải như
phải rồi
phải tay
phần
phần lớn
phần nhiều
phần nào
phần sau
phần việc
phắt
phỉ phui
phỏng
phỏng như
phỏng nước
phỏng theo
phỏng tính
phốc
phụt
phứt
qua
qua chuyện
qua khỏi
qua lại
qua lần
qua ngày
qua tay
qua thì
qua đi
quan trọng
quan trọng vấn đề
quan tâm
quay
quay bước
quay lại
quay số
quay đi
quá
quá bán
quá bộ
quá giờ
quá lời
quá mức
quá nhiều
quá tay
quá thì
quá tin
quá trình
quá tuổi
quá đáng
quá ư
quả
quả là
quả thật
quả thế
quả vậy
quận
ra
ra bài
ra bộ
ra chơi
ra gì
ra lại
ra lời
ra ngôi
ra người
ra sao
ra tay
ra vào
ra ý
ra điều
ra đây
ren rén
riu ríu
riêng
riêng từng
riệt
rày
ráo
ráo cả
ráo nước
ráo trọi
rén
rén bước
rích
rón rén
rõ
rõ là
rõ thật
rút cục
răng
răng răng
rất
rất lâu
rằng
rằng là
rốt cuộc
rốt cục
rồi
rồi nữa
rồi ra
rồi sao
rồi sau
rồi tay
rồi thì
rồi xem
rồi đây
rứa
sa sả
sang
sang năm
sang sáng
sang tay
sao
sao bản
sao bằng
sao cho
sao vậy
sao đang
sau
sau chót
sau cuối
sau cùng
sau hết
sau này
sau nữa
sau sau
sau đây
sau đó
so
so với
song le
suýt
suýt nữa
sáng
sáng ngày
sáng rõ
sáng thế
sáng ý
sì
sì sì
sất
sắp
sắp đặt
sẽ
sẽ biết
sẽ hay
số
số cho biết
số cụ thể
số loại
số là
số người
số phần
số thiếu
sốt sột
sớm
sớm ngày
sở dĩ
sử dụng
sự
sự thế
sự việc
tanh
tanh tanh
tay
tay quay
tha hồ
tha hồ chơi
tha hồ ăn
than ôi
thanh
thanh ba
thanh chuyển
thanh không
thanh thanh
thanh tính
thanh điều kiện
thanh điểm
thay đổi
thay đổi tình trạng
theo
theo bước
theo như
theo tin
thi thoảng
thiếu
thiếu gì
thiếu điểm
thoạt
thoạt nghe
thoạt nhiên
thoắt
thuần
thuần ái
thuộc
thuộc bài
thuộc cách
thuộc lại
thuộc từ
thà
thà là
thà rằng
thành ra
thành thử
thái quá
tháng
tháng ngày
tháng năm
tháng tháng
thêm
thêm chuyện
thêm giờ
thêm vào
thì
thì giờ
thì là
thì phải
thì ra
thì thôi
thình lình
thích
thích cứ
thích thuộc
thích tự
thích ý
thím
thôi
thôi việc
thúng thắng
thương ôi
thường
thường bị
thường hay
thường khi
thường số
thường sự
thường thôi
thường thường
thường tính
thường tại
thường xuất hiện
thường đến
thảo hèn
thảo nào
thấp
thấp cơ
thấp thỏm
thấp xuống
thấy
thấy tháng
thẩy
thậm
thậm chí
thậm cấp
thậm từ
thật
thật chắc
thật là
thật lực
thật quả
thật ra
thật sự
thật thà
thật tốt
thật vậy
thế
thế chuẩn bị
thế là
thế lại
thế mà
thế nào
thế nên
thế ra
thế sự
thế thì
thế thôi
thế thường
thế thế
thế à
thế đó
thếch
thỉnh thoảng
thỏm
thốc
thốc tháo
thốt
thốt nhiên
thốt nói
thốt thôi
thộc
thời gian
thời gian sử dụng
thời gian tính
thời điểm
thục mạng
thứ
thứ bản
thứ đến
thửa
thực hiện
thực hiện đúng
thực ra
thực sự
thực tế
thực vậy
tin
tin thêm
tin vào
tiếp theo
tiếp tục
tiếp đó
tiện thể
toà
toé khói
toẹt
trong
trong khi
trong lúc
trong mình
trong ngoài
trong này
trong số
trong vùng
trong đó
trong ấy
tránh
tránh khỏi
tránh ra
tránh tình trạng
tránh xa
trên
trên bộ
trên dưới
trước
trước hết
trước khi
trước kia
trước nay
trước ngày
trước nhất
trước sau
trước tiên
trước tuổi
trước đây
trước đó
trả
trả của
trả lại
trả ngay
trả trước
trếu tráo
trển
trệt
trệu trạo
trỏng
trời đất ơi
trở thành
trừ phi
trực tiếp
trực tiếp làm
tuy
tuy có
tuy là
tuy nhiên
tuy rằng
tuy thế
tuy vậy
tuy đã
tuyệt nhiên
tuần tự
tuốt luốt
tuốt tuồn tuột
tuốt tuột
tuổi
tuổi cả
tuổi tôi
tà tà
tên
tên chính
tên cái
tên họ
tên tự
tênh
tênh tênh
tìm
tìm bạn
tìm cách
tìm hiểu
tìm ra
tìm việc
tình trạng
tính
tính cách
tính căn
tính người
tính phỏng
tính từ
tít mù
tò te
tôi
tôi con
tông tốc
tù tì
tăm tắp
tăng
tăng chúng
tăng cấp
tăng giảm
tăng thêm
tăng thế
tại
tại lòng
tại nơi
tại sao
tại tôi
tại vì
tại đâu
tại đây
tại đó
tạo
tạo cơ hội
tạo nên
tạo ra
tạo ý
tạo điều kiện
tấm
tấm bản
tấm các
tấn
tấn tới
tất cả
tất cả bao nhiêu
tất thảy
tất tần tật
tất tật
tập trung
tắp
tắp lự
tắp tắp
tọt
tỏ ra
tỏ vẻ
tốc tả
tối ư
tốt
tốt bạn
tốt bộ
tốt hơn
tốt mối
tốt ngày
tột
tột cùng
tớ
tới
tới gần
tới mức
tới nơi
tới thì
tức thì
tức tốc
từ
từ căn
từ giờ
từ khi
từ loại
từ nay
từ thế
từ tính
từ tại
từ từ
từ ái
từ điều
từ đó
từ ấy
từng
từng cái
từng giờ
từng nhà
từng phần
từng thời gian
từng đơn vị
từng ấy
tự
tự cao
tự khi
tự lượng
tự tính
tự tạo
tự vì
tự ý
tự ăn
tựu trung
veo
veo veo
việc
việc gì
vung thiên địa
vung tàn tán
vung tán tàn
và
vài
vài ba
vài người
vài nhà
vài nơi
vài tên
vài điều
vào
vào gặp
vào khoảng
vào lúc
vào vùng
vào đến
vâng
vâng chịu
vâng dạ
vâng vâng
vâng ý
vèo
vèo vèo
vì
vì chưng
vì rằng
vì sao
vì thế
vì vậy
ví bằng
ví dù
ví phỏng
ví thử
vô hình trung
vô kể
vô luận
vô vàn
vùng
vùng lên
vùng nước
văng tê
vượt
vượt khỏi
vượt quá
vạn nhất
vả chăng
vả lại
vấn đề
vấn đề quan trọng
vẫn
vẫn thế
vậy
vậy là
vậy mà
vậy nên
vậy ra
vậy thì
vậy ư
về
về không
về nước
về phần
về sau
về tay
vị trí
vị tất
vốn dĩ
với
với lại
với nhau
vở
vụt
vừa
vừa khi
vừa lúc
vừa mới
vừa qua
vừa rồi
vừa vừa
xa
xa cách
xa gần
xa nhà
xa tanh
xa tắp
xa xa
xa xả
xem
xem lại
xem ra
xem số
xin
xin gặp
xin vâng
xiết bao
xon xón
xoành xoạch
xoét
xoẳn
xoẹt
xuất hiện
xuất kì bất ý
xuất kỳ bất ý
xuể
xuống
xăm xúi
xăm xăm
xăm xắm
xảy ra
xềnh xệch
xệp
xử lý
yêu cầu
à
à này
à ơi
ào
ào vào
ào ào
á
á à
ái
ái chà
ái dà
áng
áng như
âu là
ít
ít biết
ít có
ít hơn
ít khi
ít lâu
ít nhiều
ít nhất
ít nữa
ít quá
ít ra
ít thôi
ít thấy
ô hay
ô hô
ô kê
ô kìa
ôi chao
ôi thôi
ông
ông nhỏ
ông tạo
ông từ
ông ấy
ông ổng
úi
úi chà
úi dào
ý
ý chừng
ý da
ý hoặc
ăn
ăn chung
ăn chắc
ăn chịu
ăn cuộc
ăn hết
ăn hỏi
ăn làm
ăn người
ăn ngồi
ăn quá
ăn riêng
ăn sáng
ăn tay
ăn trên
ăn về
đang
đang tay
đang thì
điều
điều gì
điều kiện
điểm
điểm chính
điểm gặp
điểm đầu tiên
đành đạch
đáng
đáng kể
đáng lí
đáng lý
đáng lẽ
đáng số
đánh giá
đánh đùng
đáo để
đâu
đâu có
đâu cũng
đâu như
đâu nào
đâu phải
đâu đâu
đâu đây
đâu đó
đây
đây này
đây rồi
đây đó
đã
đã hay
đã không
đã là
đã lâu
đã thế
đã vậy
đã đủ
đó
đó đây
đúng
đúng ngày
đúng ra
đúng tuổi
đúng với
đơn vị
đưa
đưa cho
đưa chuyện
đưa em
đưa ra
đưa tay
đưa tin
đưa tới
đưa vào
đưa về
đưa xuống
đưa đến
được
được cái
được lời
được nước
được tin
đại loại
đại nhân
đại phàm
đại để
đạt
đảm bảo
đầu tiên
đầy
đầy năm
đầy phè
đầy tuổi
đặc biệt
đặt
đặt làm
đặt mình
đặt mức
đặt ra
đặt trước
đặt để
đến
đến bao giờ
đến cùng
đến cùng cực
đến cả
đến giờ
đến gần
đến hay
đến khi
đến lúc
đến lời
đến nay
đến ngày
đến nơi
đến nỗi
đến thì
đến thế
đến tuổi
đến xem
đến điều
đến đâu
đều
đều bước
đều nhau
đều đều
để
để cho
để giống
để không
để lòng
để lại
để mà
để phần
để được
để đến nỗi
đối với
đồng thời
đủ
đủ dùng
đủ nơi
đủ số
đủ điều
đủ điểm
ơ
ơ hay
ơ kìa
ơi
ơi là
ư
ạ
ạ ơi
ấy
ấy là
ầu ơ
ắt
ắt hẳn
ắt là
ắt phải
ắt thật
ối dào
ối giời
ối giời ơi
ồ
ồ ồ
ổng
ớ
ớ này
ờ
ờ ờ
ở
ở lại
ở như
ở nhờ
ở năm
ở trên
ở vào
ở đây
ở đó
ở được
ủa
ứ hự
ứ ừ
ừ
ừ nhé
ừ thì
ừ ào
ừ ừ
ử"""
VNM_stopwords = VNM.split('\n')
companyCode = company['0'].tolist()
companyName = company['1'].tolist()
VNM_stopwords.extend(companyCode)
VNM_stopwords.extend(companyName)

start_date = date_format(start) # Replace with the start date in format "mm/dd/yyyy"
end_date = date_format(end) # Replace with the start date in format "mm/dd/yyyy"
news_scrape = google_news_scraper(query,start_date,end_date,webdriver_path)
for i in range(0,len(news_scrape)):
    print("news: ",i+1)
    full_content = news_contents(news_scrape[i]['URL'],news_scrape[i]['Snippet'],query,key_search,webdriver_path)
    VNM_stopwords
    summary_content = extractive_summary(full_content,VNM_stopwords)
    news_scrape[i].update({
        "Full": full_content,
        "Summary": summary_content
    })
    time.sleep(1)
print('done')

Page Title: TCB - Tìm trên Google
Current URL: https://www.google.com/search?q=TCB&tbm=nws&tbs=cdr:1,cd_min:01/01/2023,cd_max:01/30/2023
news:  1
news:  2
news:  3
news:  4
news:  5
news:  6
news:  7
news:  8
news:  9
news:  10
done


In [3]:
dataset = [] # run to reset dataset

In [35]:
querying_date = end[3:]
news1 = news_scrape[0]['Summary']
news2 = news_scrape[1]['Summary']
news3 = news_scrape[2]['Summary']
news4 = news_scrape[3]['Summary']
news5 = news_scrape[4]['Summary']
news6 = news_scrape[5]['Summary']
news7 = news_scrape[6]['Summary']
news8 = news_scrape[7]['Summary']
news9 = news_scrape[8]['Summary']
news10 = news_scrape[9]['Summary']
dataset.append({
    "Key search": key_search,
    "Date": querying_date,
    "News 1": news1,
    "News 2": news2,
    "News 3": news3,
    "News 4": news4,
    "News 5": news5,
    "News 6": news6,
    "News 7": news7,
    "News 8": news8,
    "News 9": news9,
    "News 10": news10
})
print(key_search)
print(len(dataset))
print(start)
print(querying_date)

TCB
16
01/01/2023
01/2023


In [46]:
print(len(dataset))
print(querying_date)

20
11/2021


In [36]:
df_search = pd.DataFrame(dataset)
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"dataset_EXTRA_3_{timestamp}_len{len(dataset)}.CSV"
df_search.to_csv(filename, index=False, encoding='utf-8-sig')
print(f"Search results for News saved to '{filename}'")

Search results for Vietnam saved to 'dataset_EXTRA_3_2023-12-24_16-47-06_len16.CSV'


In [21]:
df_search = pd.DataFrame(news_scrape)
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"news_search_{timestamp}_len{len(dataset)}.CSV"
df_search.to_csv(filename, index=False, encoding='utf-8-sig')
print(f"Search results for News saved to '{filename}'")

Search results for Vietnam saved to 'news_search_2023-12-20_12-39-21_len8.CSV'


concat 2 or more datasets into 1

In [38]:
df_test_2 = pd.read_csv('dataset_90_unchanged-Copy.CSV')
df_test_3 = pd.read_csv('dataset_EXTRA_3_2023-12-24_16-47-06_len16.CSV')
df_test_result = pd.concat([df_test_2,df_test_3],ignore_index=True)

In [39]:
df_search = pd.DataFrame(df_test_result)
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"dataset.CSV"
df_search.to_csv(filename, index=False, encoding='utf-8-sig')
print(f"Search results for News saved to '{filename}'")

Search results for Vietnam saved to 'dataset.CSV'
